In [12]:
""" External Imports """
import maboss
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import shutil
import sys
import yaml
import numpy as np
sys.path.append("/Users/emilieyu/endotehelial-masboss")

In [13]:
""" Packge Imports """
from boolean_models.analysis import (
    compute_delta,
    classify_phenotype,
    save_df_to_csv
)
from boolean_models.scripts.run_perturbations import run_perturbations
from boolean_models.scripts.run_param_sweep import run_param_sweep_single


In [14]:

""" Global Setting Configuration """

PROJECT_ROOT = Path("/Users/emilieyu/endotehelial-masboss/")
CONFIG_PATH = PROJECT_ROOT / "config" / "rho_sim_config.yaml"

with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# MaBoSS Model files
MODELS_BND = PROJECT_ROOT / config['paths']['model_bnd']
MODELS_CFG = PROJECT_ROOT / config['paths']['model_cfg']

# Result directory
RESULTS_DIR = PROJECT_ROOT / config['paths']['results_base']

# Subdirectories to store parameter sweep / perturbation result
PARAM_DIR = RESULTS_DIR / config['paths']['subdirs']['param_sweep']
PERBS_DIR = RESULTS_DIR / config['paths']['subdirs']['perturbation_sim']

# Perturbations
PERBS_DICT = config['perturbations'] # get mutations directly from config
PARAM_LIST = config['sensitivity_analysis'] # get list of parameters

In [19]:
base_model = maboss.load(str(MODELS_BND), str(MODELS_CFG))
base_model.param['max_time'] = 10.0
base_model.param['sample_count'] = 5000
base_model.param

OrderedDict([('time_tick', 0.1),
             ('max_time', 10.0),
             ('sample_count', 5000),
             ('discrete_time', 0),
             ('use_physrandgen', 1),
             ('seed_pseudorandom', 0),
             ('display_traj', 0),
             ('statdist_traj_count', 0),
             ('statdist_cluster_threshold', 1),
             ('thread_count', 1),
             ('statdist_similarity_cache_max_size', 20000),
             ('$RhoA_amp', '10.0'),
             ('$RhoA_mod', '3.0'),
             ('$RhoA_basal', '0.8'),
             ('$RhoA_antagonistic', '3.0'),
             ('$RhoA_decay', '0.2'),
             ('$RhoC_amp', '10.0'),
             ('$RhoC_mod', '3.0'),
             ('$RhoC_basal', '0.8'),
             ('$RhoC_antagonistic', '3.0'),
             ('$RhoC_decay', '0.2')])

In [20]:
perb_df = run_perturbations(base_model, PERBS_DIR, config)
perb_df

DEUBG: Loaded base MaBoSS model
DEBUG: Running scenario: WT
DEBUG: Running scenario: DSP
DEBUG: Running scenario: TJP1
DEBUG: Running scenario: JCAD
DEBUG: Running scenario: DSP_JCAD
DEBUG: Running scenario: TJP1_JCAD
DEBUG: All simulations completed successfully
DEBUG: File perturbation_timeseries_05.csv successfully written to directory: /Users/emilieyu/endotehelial-masboss/results/boolean_models/perturbation_sim


,t,DSP,TJP1,JCAD,RhoA,RhoC,perturbation,delta,phenotype
0,0.0,0.501600,0.495600,0.500199,0.532183,0.542832,WT,0.010649,Normal
1,0.1,0.501600,0.495601,0.500201,0.574413,0.574765,WT,0.000352,Normal
2,0.2,0.501599,0.495598,0.500201,0.582257,0.583067,WT,0.000810,Normal
3,0.3,0.501601,0.495600,0.500201,0.586030,0.584062,WT,-0.001968,Normal
4,0.4,0.501601,0.495601,0.500202,0.589838,0.585897,WT,-0.003941,Normal
...,...,...,...,...,...,...,...,...,...
595,9.5,0.493200,0.000000,0.000000,0.650446,0.417357,TJP1_JCAD,-0.233089,Normal
596,9.6,0.493200,0.000000,0.000000,0.651849,0.414233,TJP1_JCAD,-0.237616,Normal
597,9.7,0.493200,0.000000,0.000000,0.649412,0.415500,TJP1_JCAD,-0.233912,Normal
598,9.8,0.493200,0.000000,0.000000,0.641678,0.416493,TJP1_JCAD,-0.225185,Normal


In [21]:
dsp_df = perb_df[perb_df['perturbation'] == 'DSP_JCAD']
dsp_df

,t,DSP,TJP1,JCAD,RhoA,RhoC,perturbation,delta,phenotype
400,0.0,0.0,0.495600,0.0,0.491789,0.512994,DSP_JCAD,0.021205,Normal
401,0.1,0.0,0.495600,0.0,0.465506,0.535255,DSP_JCAD,0.069749,Normal
402,0.2,0.0,0.495600,0.0,0.445909,0.553026,DSP_JCAD,0.107117,Normal
403,0.3,0.0,0.495600,0.0,0.434378,0.572896,DSP_JCAD,0.138518,Normal
404,0.4,0.0,0.495600,0.0,0.434067,0.586377,DSP_JCAD,0.152310,Normal
...,...,...,...,...,...,...,...,...,...
495,9.5,0.0,0.495600,0.0,0.414876,0.650725,DSP_JCAD,0.235849,Normal
496,9.6,0.0,0.495599,0.0,0.409781,0.649188,DSP_JCAD,0.239407,Normal
497,9.7,0.0,0.495601,0.0,0.410855,0.643094,DSP_JCAD,0.232239,Normal
498,9.8,0.0,0.495600,0.0,0.419238,0.643223,DSP_JCAD,0.223985,Normal
